In [71]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from random import shuffle
from sklearn import preprocessing
import gc, copy
from sklearn.feature_extraction import FeatureHasher

In [72]:
# Import data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
cols = ["MSZoning", "Street", "LotShape", "LandContour", "SaleType"]
data = pd.concat([df_train[cols], df_test[cols]])

#### Method 1: Encoding to ordinal variables

In [74]:
def encode_to_ordinal_var():
    for c in cols:    
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(data[c].values))
        data[c] = lbl.transform(list(data[c].values))
# encode_to_ordinal_var()
# data

#### Method 2: One hot encoding (or dummy variabales)

In [75]:
# data = pd.get_dummies(data)
# data

#### Method 3: Feature hashing (a.k.a the hashing trick)

In [76]:
def feature_hashing():
    for c in cols:
        data[c] = data[c].astype('str')
        h = FeatureHasher(n_features=5, input_type='string')
        f = h.transform(data[c].values)
        for i in range(5):
            data[c+str(i+1)] = f.toarray()[:,i]
        data.drop(columns=[c], index=1)
# feature_hashing()

#### Method 4: Cat2Vec

In [77]:
def apply_w2v(sentences, model, num_features):
    def _average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        n_words = 0.
        for word in words:
            if word in vocabulary: 
                n_words = n_words + 1.
                feature_vector = np.add(feature_vector, model[word])

        if n_words:
            feature_vector = np.divide(feature_vector, n_words)
        return feature_vector
    
    vocab = set(model.wv.index2word)
    feats = [_average_word_vectors(s, model, vocab, num_features) for s in sentences]
    return np.array(feats)

In [78]:
def gen_cat2vec_sentences(data2):
    X_w2v = copy.deepcopy(data2)
    names = list(X_w2v.columns.values)
    for c in names:
        X_w2v[c] = X_w2v[c].fillna('unknow').astype('category')
        X_w2v[c].cat.categories = ["%s %s" % (c,g) for g in X_w2v[c].cat.categories]
    X_w2v = X_w2v.values.tolist()
    return X_w2v

In [79]:
n_cat2vec_feature  = len(cols) # define the cat2vecs dimentions
n_cat2vec_window   = len(cols)*2 # define the w2v window size

In [80]:
def fit_cat2vec_model():
    X_w2v = gen_cat2vec_sentences(data.loc[:,cols].sample(frac=0.6))
    for i in X_w2v:
        shuffle(i)
    model = Word2Vec(X_w2v, size=n_cat2vec_feature, window=n_cat2vec_window)
    return model

c2v_model = fit_cat2vec_model()

In [81]:
c2v_matrix = apply_w2v(gen_cat2vec_sentences(data.loc[:,cols]), c2v_model, n_cat2vec_feature)
c2v_matrix

/home/rahul/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([[-0.57997351,  0.76226921, -0.62828134,  0.28849094,  0.33088311],
       [-0.57997351,  0.76226921, -0.62828134,  0.28849094,  0.33088311],
       [-0.52657577,  0.707407  , -0.57533414,  0.27144382,  0.3345457 ],
       ...,
       [-0.57997351,  0.76226921, -0.62828134,  0.28849094,  0.33088311],
       [-0.57997351,  0.76226921, -0.62828134,  0.28849094,  0.33088311],
       [-0.57997351,  0.76226921, -0.62828134,  0.28849094,  0.33088311]])